Импорт библиотек

In [15]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.models as M
import tensorflow.keras.layers as L
import tensorflow.keras.backend as K
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd
print("We're using TF", tf.__version__)

We're using TF 2.16.1


Данные для обучения

In [16]:
images = np.load('data/images.npy')
labels = np.load('data/labels.npy')

print(images.shape)
print(labels.shape)

(20000, 48, 48, 3)
(20000,)


In [17]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size = 0.2, random_state = 42)
display(X_train.shape)
display(X_test.shape)

(16000, 48, 48, 3)

(4000, 48, 48, 3)

Центрируем и нормируем

In [18]:
x_train2 = X_train.astype(np.float32) / 255 - 0.5
x_test2 = X_test.astype(np.float32) / 255 - 0.5

one-hot encode для ответов

In [19]:
y_train2 = keras.utils.to_categorical(y_train, 26)
y_test2 = keras.utils.to_categorical(y_test, 26)

print(y_train2.shape)
print(y_train2[:5], y_train[:5])

(16000, 26)
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
  0. 0.]] [20 22 14 14 22]


In [20]:
K.clear_session()

model = M.Sequential()
model.add(L.Conv2D(32, kernel_size = 3, padding = 'same', activation = 'relu', input_shape=(48,48,3)))
model.add(L.BatchNormalization())
model.add(L.Conv2D(32, kernel_size = 3, padding = 'same', activation = 'relu'))
model.add(L.BatchNormalization())
model.add(L.Conv2D(32, kernel_size = 3, padding = 'same', strides = 2,  activation = 'relu'))
model.add(L.BatchNormalization())
model.add(L.Dropout(rate = 0.5))
model.add(L.Conv2D(64, kernel_size = 3, padding = 'same', activation = 'relu'))
model.add(L.MaxPool2D(pool_size = 2, padding = 'same' ))
model.add(L.BatchNormalization())
model.add(L.Conv2D(64, kernel_size = 3, padding = 'same', activation = 'relu'))
model.add(L.MaxPool2D(pool_size = 2, padding = 'same' ))
model.add(L.BatchNormalization())
model.add(L.Conv2D(64, kernel_size = 3, padding = 'same', activation = 'relu'))
model.add(L.MaxPool2D(pool_size = 2, padding = 'same' ))
model.add(L.BatchNormalization())
model.add(L.Dropout(rate = 0.5))
model.add(L.Conv2D(128, kernel_size = 3, padding = 'same', activation = 'relu'))
model.add(L.MaxPool2D(pool_size = 2, padding = 'same' ))
model.add(L.BatchNormalization())
model.add(L.Flatten())
model.add(L.Dense(1024, activation='relu'))
model.add(L.Dropout(rate = 0.5))
model.add(L.Dense(512, activation='relu'))
model.add(L.Dense(26, activation = 'softmax'))

model.summary()

/Users/anastasiyachumakova/Library/Caches/pypoetry/virtualenvs/dl-hw-mfti-2-4zh9IwC0-py3.11/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 48, 48, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 48, 48, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 48, 48, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 48, 48, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 24, 24, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 24, 24, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 24, 24, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 24, 24, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 12, 12, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 12, 12, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 6, 6, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 6, 6, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 3, 3, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 3, 3, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 3, 3, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 3, 3, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 2, 2, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 2, 2, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 1,250,714 (4.77 MB)

 Trainable params: 1,249,882 (4.77 MB)

 Non-trainable params: 832 (3.25 KB)

Минимизируем кросс-энтропию и выводим процент правильных ответов

In [21]:
model.compile(
    loss='categorical_crossentropy',  
    optimizer='adam', 
    metrics=['accuracy']  
)

In [22]:
history = model.fit(
                    x_train2, 
                    y_train2,
                    batch_size=128,
                    epochs=80,
                    validation_data=(x_test2, y_test2)
                    )

Epoch 1/80
125/125 ━━━━━━━━━━━━━━━━━━━━ 28s 213ms/step - accuracy: 0.0465 - loss: 3.3792 - val_accuracy: 0.0260 - val_loss: 3.2768
Epoch 2/80
125/125 ━━━━━━━━━━━━━━━━━━━━ 26s 212ms/step - accuracy: 0.0572 - loss: 3.2120 - val_accuracy: 0.0355 - val_loss: 3.5390
Epoch 3/80
125/125 ━━━━━━━━━━━━━━━━━━━━ 27s 213ms/step - accuracy: 0.1281 - loss: 2.9277 - val_accuracy: 0.0698 - val_loss: 3.4830
Epoch 4/80
125/125 ━━━━━━━━━━━━━━━━━━━━ 28s 227ms/step - accuracy: 0.2723 - loss: 2.3817 - val_accuracy: 0.4408 - val_loss: 1.7582
Epoch 5/80
125/125 ━━━━━━━━━━━━━━━━━━━━ 27s 215ms/step - accuracy: 0.4834 - loss: 1.6310 - val_accuracy: 0.6450 - val_loss: 1.1208
Epoch 6/80
125/125 ━━━━━━━━━━━━━━━━━━━━ 29s 231ms/step - accuracy: 0.6241 - loss: 1.1686 - val_accuracy: 0.7707 - val_loss: 0.7429
Epoch 7/80
125/125 ━━━━━━━━━━━━━━━━━━━━ 28s 221ms/step - accuracy: 0.7095 - loss: 0.8863 - val_accuracy: 0.8030 - val_loss: 0.6118
Epoch 8/80
125/125 ━━━━━━━━━━━━━━━━━━━━ 26s 208ms/step - accuracy: 0.7444 - loss: 0

Прочитаем данные из файла "images_sub.npy"

In [23]:
images_sub = np.load('data/images_sub.npy')
images_sub2 = images_sub.astype(np.float32) / 255 - 0.5

In [24]:
predicts = model.predict(images_sub2)
classes_x=np.argmax(predicts,axis=1)
classes_x

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 25s 16ms/step


array([19,  8, 14, ..., 14,  3, 25])

Прочитаем пример sample_submission.csv

In [25]:
submission = pd.read_csv('data/sample_submission.csv')
submission

,Id,Category
0,0,13
1,1,20
2,2,12
3,3,7
4,4,8
...,...,...
49995,49995,15
49996,49996,4
49997,49997,16
49998,49998,8


Запишем предсказанные метки классов 

In [26]:
submission['Category'] = classes_x
submission

,Id,Category
0,0,19
1,1,8
2,2,14
3,3,12
4,4,10
...,...,...
49995,49995,4
49996,49996,0
49997,49997,14
49998,49998,3


In [27]:
submission.to_csv('data/result_submission.csv', index = False)